# Lean 9 : LeanDojo - ML/LLM Theorem Proving

## Introduction

**LeanDojo** est une bibliotheque Python revolutionnaire qui permet l'interaction programmatique avec Lean 4 pour le machine learning et la demonstration automatique de theoremes. Elle a ete introduite a NeurIPS 2023 et constitue aujourd'hui un outil essentiel pour la recherche en IA mathematique.

### Pourquoi LeanDojo ?

La demonstration automatique de theoremes (Automated Theorem Proving) est l'un des defis majeurs de l'IA. LeanDojo permet de :

| Capacite | Description | Interet pour l'IA |
|----------|-------------|-------------------|
| **Tracing** | Analyse et extraction de metadonnees d'un depot Lean | Dataset d'entrainement |
| **Theorem Extraction** | Liste tous les theoremes avec leurs types et preuves | Supervision pour ML |
| **Dojo Environment** | REPL interactif pour executer des tactiques | Reinforcement Learning |
| **Tactic State** | Acces a l'etat de preuve (goals, hypotheses) | Features pour modeles |

### Architecture de LeanDojo

```
LeanDojo
├── LeanGitRepo      # Reference a un depot Git Lean
├── trace()          # Analyse le depot et extrait les metadonnees
├── TracedRepo       # Depot analyse avec acces aux theoremes
│   └── get_theorems()  # Iterateur sur tous les theoremes
├── Theorem          # Representation d'un theoreme
│   ├── full_name    # Nom complet (ex: Nat.add_comm)
│   ├── file_path    # Fichier source
│   └── code         # Code source Lean
└── Dojo             # Environnement interactif pour prouver
    ├── run_tac()    # Execute une tactique
    └── TacticState  # Etat apres chaque tactique
        ├── goals    # Liste des buts restants
        └── pp       # Pretty-print de l'etat
```

### Prerequis

- **Python 3.10-3.12** (pas 3.13+ pour l'instant)
- **GitHub token** pour eviter le rate limiting de l'API
- **~5 Go d'espace disque** pour le cache (Mathlib4 est volumineux)
- **WSL sur Windows** (recommande pour eviter les blocages)

**Duree estimee** : 45 minutes

## Configuration Principale

Cette cellule centralise **tous les switches** pour controler l'execution du notebook.
Modifiez les valeurs selon vos besoins avant de lancer l'execution.

### Operations et temps estimes

| Operation | Switch | Temps (cache) | Temps (1ere fois) | Description |
|-----------|--------|---------------|-------------------|-------------|
| **Tracing** | `ENABLE_TRACING` | 1-2 min | 5-15 min | Compile et trace le depot Lean |
| **Dojo** | `ENABLE_DOJO_DEMO` | 1-2 min | 30+ min | Execute des tactiques interactives |
| **Preuve auto** | `ENABLE_AUTO_PROOF` | Rapide | 30+ min | Demontre la preuve automatique |

### Modes d'execution suggeres

| Mode | Config | Temps total | Usage |
|------|--------|-------------|-------|
| **Rapide** | Tout desactive | < 30 sec | Lecture seule |
| **Demo** | Tracing ON | 2-3 min | Voir extraction theoremes |
| **Complet** | Tout active | 5-35 min | Toutes les fonctionnalites |

In [ ]:
# =============================================================================
# CONFIGURATION PRINCIPALE - MODIFIEZ CES VALEURS
# =============================================================================

# ----- DEPOT A UTILISER -----
# "micro"  : lean4-example, 2 theoremes utilisateur (RECOMMANDE)
# "small"  : lean4-example, 10 premiers theoremes
# "medium" : formal-conjectures, ~100 theoremes (30-60 min 1ere fois)
# "large"  : mathlib4, >100k theoremes (2-4h 1ere fois)
REPO_SIZE = "micro"

# ----- OPERATIONS A ACTIVER -----
# Mettre True pour activer, False pour skip (mode demo)

ENABLE_TRACING = True      # Tracer le depot et extraire les theoremes
                           # Temps: 1-2 min (cache) / 5-15 min (1ere fois)

ENABLE_DOJO_DEMO = True    # Ouvrir un Dojo et executer des tactiques
                           # Temps: 1-2 min (cache) / 30+ min (peut re-tracer)

ENABLE_AUTO_PROOF = True   # Demontrer la preuve automatique avec LLM pattern
                           # Temps: Rapide si Dojo deja ouvert, sinon 30+ min

# ----- CONFIGURATION AVANCEE -----
TRACING_TIMEOUT_MINUTES = 15    # Timeout pour le tracing
DOJO_TIMEOUT_MINUTES = 45       # Timeout pour le Dojo
MAX_THEOREMS_DISPLAY = 10       # Nombre max de theoremes a afficher

# =============================================================================
# AFFICHAGE DE LA CONFIGURATION
# =============================================================================
print("=" * 70)
print("CONFIGURATION PRINCIPALE DU NOTEBOOK")
print("=" * 70)
print(f"""
  Depot        : {REPO_SIZE}
  
  Operations activees:
    - Tracing       : {'ON' if ENABLE_TRACING else 'OFF (mode demo)'}
    - Dojo demo     : {'ON' if ENABLE_DOJO_DEMO else 'OFF (mode demo)'}
    - Preuve auto   : {'ON' if ENABLE_AUTO_PROOF else 'OFF (mode demo)'}
  
  Timeouts:
    - Tracing       : {TRACING_TIMEOUT_MINUTES} min
    - Dojo          : {DOJO_TIMEOUT_MINUTES} min
""")

# Estimation du temps total
if ENABLE_TRACING and ENABLE_DOJO_DEMO:
    print("  Temps estime : 3-5 min (cache) / 30-60 min (1ere fois)")
elif ENABLE_TRACING:
    print("  Temps estime : 2-3 min (cache) / 5-15 min (1ere fois)")
else:
    print("  Temps estime : < 30 secondes (mode demo)")

print("\n" + "=" * 70)
print("[OK] Configuration prete - continuez l'execution du notebook")
print("=" * 70)

CONFIGURATION PRINCIPALE DU NOTEBOOK

  Depot        : micro

  Operations activees:
    - Tracing       : ON
    - Dojo demo     : OFF (mode demo)
    - Preuve auto   : ON

  Timeouts:
    - Tracing       : 15 min
    - Dojo          : 45 min

  Temps estime : 2-3 min (cache) / 5-15 min (1ere fois)

[OK] Configuration prete - continuez l'execution du notebook


In [2]:
# =============================================================================
# Utilitaire de timing pour mesurer les temps d'execution
# =============================================================================
import time
from contextlib import contextmanager
from datetime import datetime

class Timer:
    """Classe pour mesurer et afficher les temps d'execution."""
    
    def __init__(self):
        self.start_time = None
        self.cell_times = []
        self.notebook_start = time.time()
    
    def start(self, label=""):
        self.start_time = time.time()
        self.current_label = label
        return self
    
    def stop(self):
        if self.start_time:
            elapsed = time.time() - self.start_time
            self.cell_times.append((self.current_label, elapsed))
            return elapsed
        return 0
    
    def elapsed(self):
        if self.start_time:
            return time.time() - self.start_time
        return 0
    
    def format(self, seconds):
        if seconds < 1:
            return f"{seconds*1000:.0f}ms"
        elif seconds < 60:
            return f"{seconds:.1f}s"
        else:
            return f"{seconds/60:.1f}min"
    
    def summary(self):
        total = time.time() - self.notebook_start
        print("" + "=" * 60)
        print("RESUME DES TEMPS D'EXECUTION")
        print("=" * 60)
        for label, t in self.cell_times:
            print(f"  {label}: {self.format(t)}")
        print(f"  TOTAL: {self.format(total)}")
        print("=" * 60)

# Instance globale
timer = Timer()
print(f"[TIMER] Notebook demarre a {datetime.now().strftime('%H:%M:%S')}")

[TIMER] Notebook demarre a 00:46:04


## Details sur les Depots

Cette section explique les choix de depot disponibles (configures via `REPO_SIZE` ci-dessus).

### Pourquoi ~1500 fichiers meme pour un petit depot ?

Tout projet Lean 4 inclut la **bibliotheque standard** (`Init/`, `Std/`, etc.) qui contient ~1500 fichiers.
C'est **inevitable** - LeanDojo doit tracer toutes les dependances pour connaitre les premises utilisables.

Le parametre `user_files_only` permet de filtrer les theoremes pour ne garder que ceux du projet utilisateur
(ex: `Lean4Example.lean`) et ignorer les ~27000 theoremes de la stdlib.

### Qu'est-ce que le Dojo ?

Le **Dojo** est l'environnement interactif de LeanDojo pour executer des tactiques une par une.
C'est l'interface ideale pour le Reinforcement Learning et les LLMs.

**Note** : Le Dojo peut necessiter un re-tracing du depot, ce qui prend du temps.
Pour les gros depots, le Dojo est desactive par defaut.

In [3]:
# =============================================================================
# CONFIGURATION DES DEPOTS (utilise REPO_SIZE de la config principale)
# =============================================================================

# Configuration des depots disponibles
# NOTE: Tous les repos Lean 4 incluent ~1500 fichiers de la stdlib.
#       Le parametre user_files_only filtre les theoremes APRES le tracing.
REPOS = {
    "micro": {
        "url": "https://github.com/yangky11/lean4-example",
        "commit": "7761283d0aed994cd1c7e893786212d2a01d159e",
        "description": "lean4-example (theoremes utilisateur seulement)",
        "user_files_only": True,  # Ne garder que Lean4Example.lean
        "user_file_patterns": ["Lean4Example.lean"],  # Fichiers utilisateur
        "theorems_filter": None,  # Pas de limite (2 theoremes seulement)
        "tracing_time": "< 2 min (depuis cache)",
    },
    "small": {
        "url": "https://github.com/yangky11/lean4-example",
        "commit": "7761283d0aed994cd1c7e893786212d2a01d159e",
        "description": "lean4-example (utilisateur + quelques stdlib)",
        "user_files_only": False,
        "user_file_patterns": [],
        "theorems_filter": 10,  # Limiter a 10 pour la demo
        "tracing_time": "< 2 min (depuis cache)",
    },
    "medium": {
        "url": "https://github.com/google-deepmind/formal-conjectures",
        "commit": "ce0a081ab74d625948c44da6022992e1f9db070a",
        "description": "formal-conjectures (DeepMind)",
        "user_files_only": False,
        "user_file_patterns": [],
        "theorems_filter": 100,  # Limiter a 100
        "tracing_time": "30-60 min",
    },
    "large": {
        "url": "https://github.com/leanprover-community/mathlib4",
        "commit": "v4.15.0",
        "description": "mathlib4",
        "user_files_only": False,
        "user_file_patterns": [],
        "theorems_filter": 100,  # Limiter a 100 pour la demo
        "tracing_time": "2-4 heures",
    },
}

# Obtenir la configuration selectionnee (REPO_SIZE vient de la config principale)
SELECTED_REPO = REPOS[REPO_SIZE]
USER_FILES_ONLY = SELECTED_REPO.get("user_files_only", False)
USER_FILE_PATTERNS = SELECTED_REPO.get("user_file_patterns", [])

print("=" * 60)
print("DEPOT SELECTIONNE")
print("=" * 60)
print(f"Option: {REPO_SIZE}")
print(f"Depot: {SELECTED_REPO['description']}")
print(f"URL: {SELECTED_REPO['url']}")
print(f"Temps de tracing estime: {SELECTED_REPO['tracing_time']}")
print(f"Filtrer theoremes utilisateur: {USER_FILES_ONLY}")

if USER_FILE_PATTERNS:
    print(f"Fichiers utilisateur: {USER_FILE_PATTERNS}")
if SELECTED_REPO.get("theorems_filter"):
    print(f"Limite theoremes: {SELECTED_REPO['theorems_filter']}")

DEPOT SELECTIONNE
Option: micro
Depot: lean4-example (theoremes utilisateur seulement)
URL: https://github.com/yangky11/lean4-example
Temps de tracing estime: < 2 min (depuis cache)
Filtrer theoremes utilisateur: True
Fichiers utilisateur: ['Lean4Example.lean']


## 1. Verification de l'Environnement

Avant de commencer, verifions que notre environnement Python est compatible avec LeanDojo.

LeanDojo utilise des fonctionnalites specifiques de Python qui ne sont pas encore disponibles dans Python 3.13+. Si vous utilisez une version trop recente, vous devrez utiliser un environnement conda ou venv avec Python 3.10-3.12.

In [4]:
# =============================================================================
# Verification de la version Python
# LeanDojo necessite Python < 3.13 en raison de dependances specifiques
# =============================================================================
timer.start("Verification environnement")
import sys
import platform

print("=" * 60)
print("VERIFICATION DE L'ENVIRONNEMENT")
print("=" * 60)

print(f"\nPython version: {sys.version}")
print(f"Platform: {platform.system()} {platform.release()}")

# Verification de compatibilite
if sys.version_info >= (3, 13):
    print("\n[WARNING] LeanDojo necessite Python < 3.13")
    print("Solution: conda activate mcp-jupyter-py310")
    print("Ou: Utilisez le kernel 'Python 3 (WSL)' si disponible")
elif sys.version_info < (3, 10):
    print("\n[WARNING] LeanDojo necessite Python >= 3.10")
else:
    print(f"\n[OK] Python {sys.version_info.major}.{sys.version_info.minor} compatible avec LeanDojo")

# Verification de l'OS
if platform.system() == "Windows":
    print("\n[NOTE] Windows detecte - WSL recommande pour le tracing")
print(f"[TIMER] Verification environnement: {timer.format(timer.stop())}")

VERIFICATION DE L'ENVIRONNEMENT

Python version: 3.12.3 (main, Jan  8 2026, 11:30:50) [GCC 13.3.0]
Platform: Linux 6.6.87.2-microsoft-standard-WSL2

[OK] Python 3.12 compatible avec LeanDojo
[TIMER] Verification environnement: 1ms


### Installation de LeanDojo

Si LeanDojo n'est pas installe, decommentez et executez la cellule suivante.

**Note importante** : Sur Windows, il est fortement recommande d'installer LeanDojo dans WSL Ubuntu plutot que dans Windows natif, car le tracing peut se bloquer.

In [5]:
# =============================================================================
# Installation de LeanDojo (decommenter si necessaire)
# =============================================================================
# !pip install lean-dojo

# Sur Windows, preferez installer dans WSL:
# wsl -d Ubuntu
# pip install lean-dojo

print("Installation: Decommentez la ligne ci-dessus si necessaire")

Installation: Decommentez la ligne ci-dessus si necessaire


## 2. Configuration du Token GitHub

LeanDojo utilise l'API GitHub pour acceder aux depots. Sans token, vous etes limite a 60 requetes par heure, ce qui est insuffisant pour tracer un depot.

Nous allons configurer le token depuis plusieurs sources possibles :
1. Variable d'environnement existante
2. Fichier `.env` local
3. CLI `gh` (GitHub CLI) si installe

In [6]:
# =============================================================================
# Configuration du token GitHub
# Trois methodes de chargement dans l'ordre de priorite
# =============================================================================
timer.start("Configuration GitHub")
import os
from pathlib import Path

def load_github_token():
    """
    Charge le token GitHub depuis plusieurs sources.

    Ordre de priorite:
    1. Variable d'environnement GITHUB_ACCESS_TOKEN
    2. Fichier .env dans le repertoire du notebook
    3. Fichier .env dans le home
    4. GitHub CLI (gh auth token)

    Returns:
        bool: True si un token a ete trouve et configure
    """
    # Methode 1: Variable d'environnement existante
    if "GITHUB_ACCESS_TOKEN" in os.environ:
        print("[1] Token trouve dans GITHUB_ACCESS_TOKEN")
        return True

    # Methode 2: Fichier .env dans le repertoire du notebook
    # Note: __file__ n'existe pas dans Jupyter, on utilise le chemin connu
    notebook_dir = Path("/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean")
    env_paths = [
        notebook_dir / ".env",  # Repertoire du notebook (WSL)
        Path(".") / ".env",    # Repertoire courant
        Path.home() / ".env",   # Home
    ]
    
    for env_path in env_paths:
        if env_path.exists():
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line.startswith("GITHUB_ACCESS_TOKEN="):
                        token = line.split("=", 1)[1].strip().strip('"')
                        os.environ["GITHUB_ACCESS_TOKEN"] = token
                        print(f"[2] Token charge depuis {env_path}")
                        return True
                    elif line.startswith("GITHUB_TOKEN="):
                        token = line.split("=", 1)[1].strip().strip('"')
                        os.environ["GITHUB_ACCESS_TOKEN"] = token
                        print(f"[2] Token charge depuis {env_path} (GITHUB_TOKEN)")
                        return True

    # Methode 3: GitHub CLI
    try:
        import subprocess
        result = subprocess.run(["gh", "auth", "token"], capture_output=True, text=True, timeout=5)
        if result.returncode == 0 and result.stdout.strip():
            os.environ["GITHUB_ACCESS_TOKEN"] = result.stdout.strip()
            print("[3] Token charge depuis GitHub CLI (gh)")
            return True
    except Exception:
        pass

    return False

# Execution
print("=" * 60)
print("CONFIGURATION GITHUB")
print("=" * 60)

if load_github_token():
    token = os.environ["GITHUB_ACCESS_TOKEN"]
    print(f"Token: {token[:10]}...{token[-4:]}")
    print("[OK] GitHub token configure")
else:
    print("[WARNING] Aucun token GitHub trouve!")
    print("Pour configurer un token:")
    print("  1. Creez un token sur https://github.com/settings/tokens")
    print("  2. Ajoutez GITHUB_ACCESS_TOKEN=ghp_xxx dans .env")
    print("  3. Ou executez: gh auth login")
print(f"[TIMER] Configuration GitHub: {timer.format(timer.stop())}")

CONFIGURATION GITHUB
[2] Token charge depuis /mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean/.env (GITHUB_TOKEN)
Token: ghp_IMfYN5...NCsd
[OK] GitHub token configure
[TIMER] Configuration GitHub: 10ms


## 3. Import des Modules LeanDojo

Maintenant que l'environnement est configure, importons les modules principaux de LeanDojo.

Chaque module a un role specifique :
- `LeanGitRepo` : Reference a un depot Git (ne telecharge rien)
- `trace` : Fonction qui analyse un depot
- `is_available_in_cache` : Verifie si un depot est deja trace
- `Dojo` : Environnement interactif de preuve
- `Theorem` / `TacticState` : Types de donnees

In [7]:
# =============================================================================
# Import des modules LeanDojo
# =============================================================================
timer.start("Import LeanDojo")
try:
    import lean_dojo
    print(f"LeanDojo version: {lean_dojo.__version__}")

    from lean_dojo import (
        LeanGitRepo,           # Reference a un depot Lean
        trace,                 # Fonction de tracing
        is_available_in_cache, # Verifie le cache
        Dojo,                  # Environnement interactif
        Theorem,               # Representation d'un theoreme
        TacticState,           # Etat de preuve
    )

    print("\nModules importes:")
    print("  - LeanGitRepo: Reference a un depot Git")
    print("  - trace: Analyse un depot et extrait les metadonnees")
    print("  - is_available_in_cache: Verifie si deja en cache")
    print("  - Dojo: Environnement interactif de preuve")
    print("  - Theorem, TacticState: Types de donnees")
    print("\n[OK] Tous les imports reussis")

    LEANDOJO_AVAILABLE = True

except ImportError as e:
    print(f"[ERROR] Import echoue: {e}")
    print("\nSolution: pip install lean-dojo")
    LEANDOJO_AVAILABLE = False
print(f"[TIMER] Import LeanDojo: {timer.format(timer.stop())}")

/home/jesse/.lean4-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-27 00:46:04,913	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


LeanDojo version: 4.20.0

Modules importes:
  - LeanGitRepo: Reference a un depot Git
  - trace: Analyse un depot et extrait les metadonnees
  - is_available_in_cache: Verifie si deja en cache
  - Dojo: Environnement interactif de preuve
  - Theorem, TacticState: Types de donnees

[OK] Tous les imports reussis
[TIMER] Import LeanDojo: 1.1s


## 4. Creation d'une Reference de Depot

`LeanGitRepo` represente une reference a un depot Git Lean. **Il ne telecharge rien** - c'est simplement un pointeur vers un commit specifique.

### Pourquoi un commit specifique ?

LeanDojo necessite un commit precis (pas une branche) car :
1. **Reproductibilite** : Le meme commit donne toujours les memes resultats
2. **Cache** : Le cache est indexe par commit
3. **Compatibilite** : Le fichier `lean-toolchain` determine la version de Lean

### Le depot `lean4-example`

Nous utilisons le depot officiel `lean4-example` comme exemple. C'est un petit depot (~10 theoremes) ideal pour tester LeanDojo sans attendre longtemps.

In [8]:
# =============================================================================
# Creation d'une reference de depot
# lean4-example est le depot de test officiel de LeanDojo
# =============================================================================
timer.start("Creation repo reference")

# Utiliser la configuration selectionnee
EXAMPLE_REPO = {
    "url": SELECTED_REPO["url"],
    "commit": SELECTED_REPO["commit"],
    "description": SELECTED_REPO["description"],
}

print("=" * 60)
print("DEPOT DE REFERENCE")
print("=" * 60)

if LEANDOJO_AVAILABLE:
    # Creation de la reference (ne telecharge rien)
    repo = LeanGitRepo(EXAMPLE_REPO["url"], EXAMPLE_REPO["commit"])

    print(f"\nURL: {repo.url}")
    print(f"Commit: {repo.commit[:12]}...")
    print(f"Description: {EXAMPLE_REPO['description']}")

    # Verifications
    print(f"\nVerifications:")
    print(f"  Existe sur GitHub: {repo.exists()}")
    print(f"  En cache local: {is_available_in_cache(repo)}")
else:
    print("[SKIP] LeanDojo non disponible")
    repo = None
print(f"[TIMER] Creation repo reference: {timer.format(timer.stop())}")

DEPOT DE REFERENCE

URL: https://github.com/yangky11/lean4-example
Commit: 7761283d0aed...
Description: lean4-example (theoremes utilisateur seulement)

Verifications:
  Existe sur GitHub: True
  En cache local: True
[TIMER] Creation repo reference: 1.1s


## 5. Le Tracing : Coeur de LeanDojo

Le **tracing** est l'operation centrale de LeanDojo. C'est un processus qui :

1. **Clone** le depot Git
2. **Telecharge** les dependances (Mathlib4, Lake, etc.)
3. **Compile** le projet avec instrumentation
4. **Extrait** les metadonnees (theoremes, tactiques, etats)

### Temps de tracing estimes

| Depot | Premier tracing | Depuis cache |
|-------|-----------------|--------------|
| `lean4-example` | 5-15 min | < 1 sec |
| `formal-conjectures` | 30-60 min | < 5 sec |
| Mathlib4 | 2-4 heures | < 30 sec |

### Problemes connus sur Windows

LeanDojo utilise des subprocess qui peuvent se bloquer sur Windows. **Solutions** :

1. **Kernel WSL** : Utilisez le kernel "Python 3 (WSL)" dans Jupyter
2. **Mode SKIP** : Desactivez le tracing pour la demo
3. **Terminal** : Lancez le tracing dans un terminal separe pour voir les logs

La cellule suivante permet de configurer le comportement.

In [9]:
# =============================================================================
# CONFIGURATION DU TRACING (utilise ENABLE_TRACING de la config principale)
# =============================================================================
import time
from pathlib import Path

# Derivation depuis la config principale
# ENABLE_TRACING=True  -> SKIP_TRACING=False -> tracing reel
# ENABLE_TRACING=False -> SKIP_TRACING=True  -> mode demo
SKIP_TRACING = not ENABLE_TRACING

print("=" * 60)
print("CONFIGURATION TRACING")
print("=" * 60)
print(f"\nENABLE_TRACING: {ENABLE_TRACING}")
print(f"SKIP_TRACING: {SKIP_TRACING}")
print(f"TRACING_TIMEOUT: {TRACING_TIMEOUT_MINUTES} minutes")

if SKIP_TRACING:
    print("\n[MODE DEMO] Le tracing est desactive.")
    print("Les cellules d'extraction de theoremes afficheront des exemples.")
    print("\nPour activer: mettez ENABLE_TRACING = True dans la config principale.")
else:
    print("\n[TRACING ACTIF] Le depot sera trace.")
    print("Si deja en cache: chargement rapide (1-2 min)")
    print("Sinon: tracing complet (5-15 min pour lean4-example)")

CONFIGURATION TRACING

ENABLE_TRACING: True
SKIP_TRACING: False
TRACING_TIMEOUT: 15 minutes

[TRACING ACTIF] Le depot sera trace.
Si deja en cache: chargement rapide (1-2 min)
Sinon: tracing complet (5-15 min pour lean4-example)


### Verification du Cache

Avant de lancer un tracing, verifions l'etat du cache LeanDojo. Le cache se trouve dans `~/.cache/lean_dojo/` et contient les depots deja traces.

Si le depot est en cache, le chargement est quasi-instantane.

In [10]:
# =============================================================================
# Verification du cache LeanDojo
# =============================================================================
timer.start("Verification cache")
print("=" * 60)
print("VERIFICATION DU CACHE")
print("=" * 60)

cache_dir = Path.home() / ".cache" / "lean_dojo"
print(f"\nRepertoire cache: {cache_dir}")
print(f"Existe: {cache_dir.exists()}")

if cache_dir.exists():
    try:
        cache_contents = list(cache_dir.iterdir())
        print(f"Contenu: {len(cache_contents)} elements")

        if cache_contents:
            print("\nElements en cache:")
            for item in cache_contents[:5]:
                if item.is_dir():
                    size = sum(f.stat().st_size for f in item.rglob('*') if f.is_file()) / 1024 / 1024
                    print(f"  - {item.name} ({size:.1f} MB)")
                else:
                    print(f"  - {item.name}")
            if len(cache_contents) > 5:
                print(f"  ... et {len(cache_contents) - 5} autres")
    except PermissionError:
        print("  (acces refuse)")
else:
    print("  Cache vide - le premier tracing sera plus long")

# Verifier si notre repo est en cache
if LEANDOJO_AVAILABLE and repo:
    in_cache = is_available_in_cache(repo)
    print(f"\nlean4-example en cache: {in_cache}")
print(f"[TIMER] Verification cache: {timer.format(timer.stop())}")

VERIFICATION DU CACHE

Repertoire cache: /home/jesse/.cache/lean_dojo
Existe: True
Contenu: 2 elements

Elements en cache:
  - repos (4452.5 MB)
  - yangky11-lean4-example-7761283d0aed994cd1c7e893786212d2a01d159e (5254.9 MB)

lean4-example en cache: True
[TIMER] Verification cache: 363ms


### Execution du Tracing

La cellule suivante execute le tracing si `SKIP_TRACING = False`.

**Attention** : Sur Windows natif, le tracing peut se bloquer. Si ca prend plus de 15 minutes, interrompez le kernel et passez en mode SKIP ou utilisez WSL.

In [11]:
# =============================================================================
# Execution du tracing
# =============================================================================
timer.start("Tracing")
print("=" * 60)
print("TRACING")
print("=" * 60)

traced_repo = None
theorems = []

if not LEANDOJO_AVAILABLE or repo is None:
    print("\n[SKIP] LeanDojo non disponible")

elif SKIP_TRACING:
    print("\n[MODE DEMO] Tracing desactive (SKIP_TRACING=True)")
    print("\nPour activer le tracing reel:")
    print("  1. Mettez # Herite de la configuration - mettre True pour skip manuel SKIP_TRACING = False dans la cellule precedente")
    print("  2. Sur Windows, utilisez le kernel 'Python 3 (WSL)'")
    print("  3. Relancez les cellules depuis le debut")

elif is_available_in_cache(repo):
    print("\n[CACHE HIT] Depot deja en cache!")
    print("Chargement depuis le cache...")

    start = time.time()
    traced_repo = trace(repo)
    elapsed = time.time() - start

    print(f"\nCharge en {elapsed:.1f}s")
    print(f"Path: {traced_repo.root_dir}")

else:
    print(f"\n[TRACING] Premier tracing de {repo.url}")
    print(f"Ceci peut prendre 5-15 minutes...")
    print(f"\nSi le tracing se bloque (>15 min), interrompez et:")
    print("  - Mettez SKIP_TRACING = True")
    print("  - Ou utilisez WSL")

    start = time.time()
    traced_repo = trace(repo)
    elapsed = time.time() - start

    print(f"\nTracing termine en {elapsed/60:.1f} minutes")
    print(f"Path: {traced_repo.root_dir}")

# Resume
print("\n" + "=" * 60)
if traced_repo:
    print("[OK] traced_repo disponible - les cellules suivantes fonctionneront")
else:
    print("[INFO] traced_repo = None - les cellules suivantes seront en mode demo")
print(f"[TIMER] Tracing: {timer.format(timer.stop())}")

TRACING

[CACHE HIT] Depot deja en cache!
Chargement depuis le cache...


2026-01-27 00:46:07.282 | INFO     | lean_dojo.data_extraction.trace:trace:248 - Loading the traced repo from /home/jesse/.cache/lean_dojo/yangky11-lean4-example-7761283d0aed994cd1c7e893786212d2a01d159e/lean4-example
2026-01-27 00:46:09,830	INFO worker.py:1998 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/home/jesse/.lean4-venv/lib/python3.12/site-packages/ray/_private/worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
100%|██████████| 1518/1518 [00:54<00:00, 27.73it/s] 



Charge en 78.4s
Path: /home/jesse/.cache/lean_dojo/yangky11-lean4-example-7761283d0aed994cd1c7e893786212d2a01d159e/lean4-example

[OK] traced_repo disponible - les cellules suivantes fonctionneront
[TIMER] Tracing: 1.3min


## 6. Extraction des Theoremes

Une fois le depot trace, nous pouvons extraire tous les theoremes. Chaque theoreme contient :

- `full_name` : Nom complet (ex: `Nat.add_comm`)
- `file_path` : Chemin du fichier source
- `code` : Code source Lean

### Filtrage des theoremes

Le depot `lean4-example` contient **~27000 theoremes** au total, mais la plupart viennent de la 
**bibliotheque standard Lean** (`Init/`, `Std/`, etc.).

Avec l'option `user_files_only=True`, nous filtrons pour ne garder que les theoremes du fichier
utilisateur (`Lean4Example.lean`), soit **2 theoremes** : `hello_world` et `foo`.

| Mode | Theoremes | Description |
|------|-----------|-------------|
| `micro` | 2 | Uniquement `Lean4Example.lean` |
| `small` | 10 | Premiers theoremes (mix user + stdlib) |
| `medium/large` | 100+ | Selon depot |

In [12]:
# =============================================================================
# Extraction des theoremes depuis le depot trace
# =============================================================================
timer.start("Extraction theoremes")
print("=" * 60)
print("EXTRACTION DES THEOREMES")
print("=" * 60)

def is_user_file(file_path, patterns):
    """Verifie si le fichier correspond aux patterns utilisateur."""
    file_str = str(file_path)
    for pattern in patterns:
        if pattern in file_str:
            return True
    return False

def is_stdlib_file(file_path):
    """Verifie si le fichier fait partie de la stdlib Lean."""
    file_str = str(file_path)
    stdlib_patterns = [
        "src/lean/",      # Lean core
        "Init/",          # Init module
        "Std/",           # Std library
        "Lake/",          # Lake build system
        "Lean/",          # Lean module
        ".lake/",         # Lake cache
    ]
    for pattern in stdlib_patterns:
        if pattern in file_str:
            return True
    return False

if traced_repo is None:
    print("\n[MODE DEMO] Tracing non effectue")
    print("Les theoremes ne sont pas disponibles.")
    print("\nExemple de ce que vous verriez avec un tracing reel:")
    print("  Found 2 user theorems (filtered from 27360)")
    print("  [1] hello_world (Lean4Example.lean)")
    print("  [2] foo (Lean4Example.lean)")

    theorems = []
else:
    # Extraction de TOUS les theoremes
    all_theorems = list(traced_repo.get_traced_theorems())
    total_count = len(all_theorems)
    
    # Appliquer le filtre utilisateur si demande
    if USER_FILES_ONLY and USER_FILE_PATTERNS:
        print(f"[FILTER] Filtrage par fichiers utilisateur: {USER_FILE_PATTERNS}")
        theorems = [
            thm for thm in all_theorems
            if is_user_file(
                thm.theorem.file_path if hasattr(thm, 'theorem') else getattr(thm, 'file_path', ''),
                USER_FILE_PATTERNS
            )
        ]
        print(f"[OK] {len(theorems)} theoremes utilisateur (sur {total_count} total)")
    elif SELECTED_REPO.get("theorems_filter"):
        # Limiter aux N premiers
        limit = SELECTED_REPO["theorems_filter"]
        theorems = all_theorems[:limit]
        print(f"[OK] {len(theorems)} theoremes (limite a {limit} sur {total_count})")
    else:
        theorems = all_theorems
        print(f"[OK] {len(theorems)} theoremes")
    
    print(f"\n[INFO] Total dans le depot: {total_count} theoremes")
    print(f"[INFO] Dont ~{total_count - 2} de la stdlib Lean (Init/, Std/, etc.)")
    print(f"[INFO] Theoremes selectionnes: {len(theorems)}")

    # Inspecter les attributs du premier theoreme
    if theorems:
        sample = theorems[0]
        print(f"\nType: {type(sample).__name__}")
        attrs = [a for a in dir(sample) if not a.startswith('_')]
        print(f"Attributs disponibles: {attrs[:10]}...")

    # Affichage (limiter a 10 pour eviter spam)
    print("\nListe des theoremes selectionnes:")
    for i, thm in enumerate(theorems[:10]):
        # TracedTheorem a un attribut 'theorem' qui contient le Theorem
        if hasattr(thm, 'theorem'):
            name = thm.theorem.full_name if hasattr(thm.theorem, 'full_name') else str(thm.theorem)
            file_path = thm.theorem.file_path if hasattr(thm.theorem, 'file_path') else 'N/A'
        else:
            name = getattr(thm, 'full_name', getattr(thm, 'name', str(thm)))
            file_path = getattr(thm, 'file_path', 'N/A')
        print(f"  [{i+1}] {name}")
        print(f"      Fichier: {file_path}")

    if len(theorems) > 10:
        print(f"  ... et {len(theorems) - 10} autres")
print(f"[TIMER] Extraction theoremes: {timer.format(timer.stop())}")

EXTRACTION DES THEOREMES
[FILTER] Filtrage par fichiers utilisateur: ['Lean4Example.lean']
[OK] 2 theoremes utilisateur (sur 27360 total)

[INFO] Total dans le depot: 27360 theoremes
[INFO] Dont ~27358 de la stdlib Lean (Init/, Std/, etc.)
[INFO] Theoremes selectionnes: 2

Type: TracedTheorem
Attributs disponibles: ['ast', 'comments', 'end', 'file_path', 'get_num_tactics', 'get_premise_full_names', 'get_proof_node', 'get_tactic_proof', 'get_theorem_statement', 'get_traced_tactics']...

Liste des theoremes selectionnes:
  [1] hello_world
      Fichier: Lean4Example.lean
  [2] foo
      Fichier: Lean4Example.lean
[TIMER] Extraction theoremes: 4.2s


### Details d'un Theoreme

Examinons de plus pres un theoreme. L'objet `Theorem` contient toutes les informations necessaires pour travailler avec ce theoreme dans un Dojo.

In [13]:
# =============================================================================
# Exploration d'un theoreme en detail
# =============================================================================
timer.start("Details theoreme")
print("=" * 60)
print("DETAILS D'UN THEOREME")
print("=" * 60)

if not theorems:
    print("\n[MODE DEMO] Pas de theoremes disponibles")
    print("\nExemple de structure d'un theoreme:")
    print('''
    Theorem: Nat.add_comm
    File: Mathlib/Nat/Basic.lean
    Module: Nat.Basic

    Code:
    theorem add_comm (n m : Nat) : n + m = m + n := by
      induction n with
      | zero => simp
      | succ n ih => simp [Nat.succ_add, ih]
    ''')
else:
    traced_thm = theorems[0]
    
    # TracedTheorem wraps Theorem - get the inner theorem
    thm = traced_thm.theorem if hasattr(traced_thm, 'theorem') else traced_thm
    name = thm.full_name if hasattr(thm, 'full_name') else str(thm)
    file_path = thm.file_path if hasattr(thm, 'file_path') else 'N/A'

    print(f"\nTheoreme: {name}")
    print(f"Fichier: {file_path}")
    if hasattr(file_path, 'stem'):
        print(f"Module: {file_path.stem}")

    # Attributs disponibles sur TracedTheorem
    print(f"\nAttributs TracedTheorem: {[a for a in dir(traced_thm) if not a.startswith('_')][:8]}")

    # Code source (si disponible)
    if hasattr(thm, 'code') and thm.code:
        print(f"\nCode source:")
        print("-" * 40)
        print(thm.code[:500])
        print("-" * 40)
print(f"[TIMER] Details theoreme: {timer.format(timer.stop())}")

DETAILS D'UN THEOREME

Theoreme: hello_world
Fichier: Lean4Example.lean
Module: Lean4Example

Attributs TracedTheorem: ['ast', 'comments', 'end', 'file_path', 'get_num_tactics', 'get_premise_full_names', 'get_proof_node', 'get_tactic_proof']
[TIMER] Details theoreme: 0ms


## 7. Environnement Dojo : Preuves Interactives

Le **Dojo** est l'environnement interactif de LeanDojo. C'est l'interface ideale pour :

- Les **LLMs** qui generent des preuves tactique par tactique
- Le **Reinforcement Learning** ou chaque tactique est une action
- L'**exploration** manuelle de preuves

### Fonctionnement du Dojo

```python
with Dojo(theorem) as (dojo, initial_state):
    # initial_state contient le but initial

    # Executer une tactique
    next_state = dojo.run_tac(current_state, "intro")

    # Si next_state is None -> tactique invalide
    # Si next_state.goals == [] -> preuve terminee
```

### Etats de Preuve (TacticState)

Un `TacticState` contient :
- `goals` : Liste des buts restants a prouver
- `pp` : Pretty-print de l'etat (format lisible)

In [14]:
# =============================================================================
# Ouverture d'un Dojo pour un theoreme
# Utilise ENABLE_DOJO_DEMO de la config principale
# =============================================================================
timer.start("Dojo ouverture")
print("=" * 60)
print("ENVIRONNEMENT DOJO")
print("=" * 60)

# Derivation depuis la config principale
SKIP_DOJO = not ENABLE_DOJO_DEMO

print(f"\nENABLE_DOJO_DEMO: {ENABLE_DOJO_DEMO}")
print(f"SKIP_DOJO: {SKIP_DOJO}")

if not theorems:
    print("\n[MODE DEMO] Pas de theoremes disponibles")
    print("Raison: Tracing desactive (ENABLE_TRACING=False)")
    print("\nExemple d'utilisation du Dojo:")
    print('''
    with Dojo(theorem) as (dojo, init_state):
        print(f"Goals: {len(init_state.goals)}")
        print(f"Goal 0: {init_state.goals[0]}")

        # Etat pretty-print
        print(init_state.pp)
        # Output: n m : Nat
        #         |- n + m = m + n
    ''')
elif SKIP_DOJO:
    print("\n[MODE DEMO] Dojo desactive (ENABLE_DOJO_DEMO=False)")
    print("Raison: Le Dojo peut re-tracer le repo (~30 minutes)")
    print("\nPour activer: mettez ENABLE_DOJO_DEMO = True dans la config principale.")
    
    traced_thm = theorems[0]
    thm = traced_thm.theorem if hasattr(traced_thm, 'theorem') else traced_thm
    name = thm.full_name if hasattr(thm, 'full_name') else str(thm)
    print(f"\nTheoreme qui serait utilise: {name}")
    
    print("\nExemple de sortie attendue:")
    print("  Etat initial:")
    print("    Nombre de buts: 1")
    print("    But 0: |- 1 + 1 = 2")
else:
    traced_thm = theorems[0]
    thm = traced_thm.theorem if hasattr(traced_thm, 'theorem') else traced_thm
    name = thm.full_name if hasattr(thm, 'full_name') else str(thm)
    print(f"\n[DOJO ACTIF] Ouverture pour: {name}")
    print("[INFO] Ceci peut prendre quelques minutes (possible re-tracing)")

    with Dojo(thm) as (dojo, init_state):
        print(f"\nEtat initial:")
        print(f"  Nombre de buts: {len(init_state.goals)}")

        if init_state.goals:
            print(f"  But 0: {init_state.goals[0]}")

        pp_state = getattr(init_state, 'pp', str(init_state))
        print(f"\n  Pretty-print:")
        for line in str(pp_state).split('\n'):
            print(f"    {line}")
        
        # Sauvegarder le dojo et l'etat pour les cellules suivantes
        DOJO_INSTANCE = dojo
        DOJO_INITIAL_STATE = init_state
        
print(f"\n[TIMER] Dojo ouverture: {timer.format(timer.stop())}")

ENVIRONNEMENT DOJO

ENABLE_DOJO_DEMO: False
SKIP_DOJO: True

[MODE DEMO] Dojo desactive (ENABLE_DOJO_DEMO=False)
Raison: Le Dojo peut re-tracer le repo (~30 minutes)

Pour activer: mettez ENABLE_DOJO_DEMO = True dans la config principale.

Theoreme qui serait utilise: hello_world

Exemple de sortie attendue:
  Etat initial:
    Nombre de buts: 1
    But 0: |- 1 + 1 = 2

[TIMER] Dojo ouverture: 1ms


### Execution de Tactiques

Dans un Dojo, nous pouvons executer des tactiques une par une et observer l'evolution de l'etat de preuve.

Une tactique peut :
- **Reussir** : Retourne un nouvel etat
- **Echouer** : Retourne `None`
- **Terminer la preuve** : Retourne un etat avec `goals == []`

In [15]:
# =============================================================================
# Execution de tactiques dans un Dojo
# Utilise ENABLE_DOJO_DEMO de la config principale
# =============================================================================
timer.start("Execution tactiques")
print("=" * 60)
print("EXECUTION DE TACTIQUES")
print("=" * 60)

# SKIP_DOJO est deja defini dans la cellule precedente

if not theorems:
    print("\n[MODE DEMO] Pas de theoremes disponibles")
    print("Raison: Tracing desactive (ENABLE_TRACING=False)")
    print("\nExemple d'execution de tactiques:")
    print('''
    Theorem: Nat.add_comm
    Initial goals: 1

    'intro n' succeeded:
      Goals remaining: 1
      New goal: |- forall m, n + m = m + n
    ''')
elif SKIP_DOJO:
    print("\n[MODE DEMO] Execution tactiques desactivee (ENABLE_DOJO_DEMO=False)")
    print("\nExemple de sortie attendue:")
    print('''
    Theorem: hello_world
    Buts initiaux: 1

    'rfl' [OK]
      Buts restants: 0
    
    [OK] Preuve terminee!
    ''')
else:
    traced_thm = theorems[0]
    thm = traced_thm.theorem if hasattr(traced_thm, 'theorem') else traced_thm
    name = thm.full_name if hasattr(thm, 'full_name') else str(thm)
    print(f"\nTheoreme: {name}")

    with Dojo(thm) as (dojo, init_state):
        print(f"Buts initiaux: {len(init_state.goals)}")

        tactics_to_try = ["intro", "intros", "rfl", "simp", "trivial", "omega"]
        current_state = init_state

        for tactic in tactics_to_try:
            if current_state is None or not current_state.goals:
                print(f"\n[OK] Preuve terminee!")
                break

            result = dojo.run_tac(current_state, tactic)

            if result is not None:
                print(f"\n'{tactic}' [OK]")
                print(f"  Buts restants: {len(result.goals)}")
                if result.goals:
                    goal_str = str(result.goals[0])[:60]
                    print(f"  Nouveau but: {goal_str}...")
                current_state = result
            else:
                print(f"'{tactic}' [FAIL] (tactique invalide)")
                
print(f"\n[TIMER] Execution tactiques: {timer.format(timer.stop())}")

EXECUTION DE TACTIQUES

[MODE DEMO] Execution tactiques desactivee (ENABLE_DOJO_DEMO=False)

Exemple de sortie attendue:

    Theorem: hello_world
    Buts initiaux: 1

    'rfl' [OK]
      Buts restants: 0

    [OK] Preuve terminee!
    

[TIMER] Execution tactiques: 0ms


## 8. Interface Simplifiee : lean_runner.py

Le module `lean_runner.py` fourni dans ce repertoire offre une interface simplifiee pour LeanDojo.

### Fonctionnalites

| Methode | Description |
|---------|-------------|
| `trace_repo(url, commit)` | Trace un depot Git |
| `get_theorems()` | Retourne la liste des theoremes |
| `prove_with_tactics(thm, tactics)` | Tente de prouver avec une sequence |

### Backend LeanDojo

Le runner peut utiliser LeanDojo comme backend pour les operations avancees.

In [16]:
# =============================================================================
# Import et configuration de lean_runner
# =============================================================================
timer.start("Lean Runner")
import sys
import os

# Ajouter le repertoire du notebook au path pour trouver lean_runner.py
notebook_dir = "/mnt/d/dev/CoursIA/MyIA.AI.Notebooks/SymbolicAI/Lean"
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)
os.chdir(notebook_dir)  # Changer le repertoire de travail

print("=" * 60)
print("LEAN RUNNER")
print("=" * 60)

try:
    from lean_runner import LeanRunner

    # Creation du runner avec backend LeanDojo
    runner = LeanRunner(backend='leandojo')
    print(f"[OK] LeanRunner initialise")
    print(f"Backend: {runner.backend}")

    RUNNER_AVAILABLE = True

except Exception as e:
    print(f"[WARNING] LeanRunner non disponible: {e}")
    print("Les cellules suivantes utilisant runner seront en mode demo")
    runner = None
    RUNNER_AVAILABLE = False
print(f"[TIMER] Lean Runner: {timer.format(timer.stop())}")

LEAN RUNNER
[OK] LeanRunner initialise
Backend: Backend.LEANDOJO
[TIMER] Lean Runner: 10ms


### Tracing via lean_runner

Le runner utilise le cache LeanDojo, donc si le depot est deja trace, c'est instantane.

In [17]:
# =============================================================================
# Tracing via lean_runner
# NOTE: Section optionnelle - on a deja trace via LeanDojo directement
# =============================================================================
print("=" * 60)
print("TRACING VIA LEAN_RUNNER")
print("=" * 60)

# Le tracing a deja ete fait via LeanDojo directement (section 5)
# Cette section montre comment le faire via lean_runner

if not RUNNER_AVAILABLE:
    print("\n[SKIP] LeanRunner non disponible")
    print("Le tracing a deja ete effectue via LeanDojo directement.")
else:
    print("\n[INFO] LeanRunner disponible")
    print("Le tracing a deja ete effectue dans la section 5.")
    print(f"traced_repo disponible: {traced_repo is not None}")
    
    if traced_repo:
        print(f"Path: {traced_repo.root_dir}")
        print(f"Theoremes selectionnes: {len(theorems)}")
        if USER_FILES_ONLY:
            print(f"Mode: Fichiers utilisateur uniquement ({USER_FILE_PATTERNS})")

TRACING VIA LEAN_RUNNER

[INFO] LeanRunner disponible
Le tracing a deja ete effectue dans la section 5.
traced_repo disponible: True
Path: /home/jesse/.cache/lean_dojo/yangky11-lean4-example-7761283d0aed994cd1c7e893786212d2a01d159e/lean4-example
Theoremes selectionnes: 2
Mode: Fichiers utilisateur uniquement (['Lean4Example.lean'])


### Extraction et Preuve via lean_runner

Une fois le depot trace, nous pouvons extraire les theoremes et tenter des preuves.

In [18]:
# =============================================================================
# Extraction des theoremes via lean_runner
# NOTE: Section optionnelle - les theoremes sont deja disponibles via traced_repo
# =============================================================================
timer.start("Extraction theoremes lean_runner")
print("=" * 60)
print("THEOREMES VIA LEAN_RUNNER")
print("=" * 60)

# Cette section est optionnelle car on a deja les theoremes via traced_repo
# Le lean_runner offre une interface simplifiee mais n'est pas necessaire

if not RUNNER_AVAILABLE:
    print("\n[SKIP] LeanRunner non disponible")
    print("Ce n'est pas grave - les theoremes sont deja extraits via traced_repo")
    thms = []
else:
    print("\n[INFO] LeanRunner disponible mais non utilise ici")
    print("Les theoremes sont deja disponibles via la variable 'theorems'")
    print(f"Nombre de theoremes selectionnes: {len(theorems) if theorems else 0}")
    if USER_FILES_ONLY and theorems:
        print(f"Mode: Fichiers utilisateur uniquement")
    thms = []  # On utilise 'theorems' directement, pas besoin de thms

print(f"\n[TIMER] Extraction theoremes lean_runner: {timer.format(timer.stop())}")

THEOREMES VIA LEAN_RUNNER

[INFO] LeanRunner disponible mais non utilise ici
Les theoremes sont deja disponibles via la variable 'theorems'
Nombre de theoremes selectionnes: 2
Mode: Fichiers utilisateur uniquement

[TIMER] Extraction theoremes lean_runner: 1ms


### Tentative de Preuve Automatique

La methode `prove_with_tactics` essaie une sequence de tactiques et retourne le resultat.

In [19]:
# =============================================================================
# Preuve automatique avec une sequence de tactiques
# Utilise ENABLE_AUTO_PROOF de la config principale
# =============================================================================
timer.start("Preuve automatique")
print("=" * 60)
print("PREUVE AUTOMATIQUE")
print("=" * 60)

print(f"\nENABLE_AUTO_PROOF: {ENABLE_AUTO_PROOF}")

# La preuve automatique necessite le Dojo
SKIP_AUTO_PROOF = not ENABLE_AUTO_PROOF or not theorems or SKIP_DOJO

if not ENABLE_AUTO_PROOF:
    print("\n[MODE DEMO] Preuve automatique desactivee (ENABLE_AUTO_PROOF=False)")
    print("\nPour activer: mettez ENABLE_AUTO_PROOF = True dans la config principale.")
elif not theorems:
    print("\n[MODE DEMO] Pas de theoremes disponibles")
    print("Raison: Tracing desactive (ENABLE_TRACING=False)")
elif SKIP_DOJO:
    print("\n[MODE DEMO] Preuve automatique desactivee (ENABLE_DOJO_DEMO=False)")
    print("Raison: Le Dojo est necessaire pour la preuve automatique")

if SKIP_AUTO_PROOF:
    print("\nPattern de preuve automatique avec LeanRunner:")
    print('''
    # Avec LeanRunner et Dojo actif:
    result = runner.prove_with_tactics(
        theorem,
        ["intro", "intros", "rfl", "simp", "omega"]
    )
    
    print(f"Succes: {result['success']}")
    for step in result['steps']:
        print(f"  {step['tactic']}: {'OK' if step['success'] else 'FAIL'}")
    ''')
    
    print("\nExemple de sortie attendue:")
    print('''
    Theorem: hello_world
    Succes: True
    Steps: 1
      rfl: OK
    ''')
else:
    # Execution reelle de la preuve automatique
    traced_thm = theorems[0]
    thm = traced_thm.theorem if hasattr(traced_thm, 'theorem') else traced_thm
    name = thm.full_name if hasattr(thm, 'full_name') else str(thm)
    print(f"\n[PREUVE ACTIVE] Theoreme: {name}")
    
    tactics_sequence = ["intro", "intros", "rfl", "simp", "trivial", "omega", "decide"]
    
    with Dojo(thm) as (dojo, init_state):
        print(f"Buts initiaux: {len(init_state.goals)}")
        
        current_state = init_state
        proof_steps = []
        
        for tactic in tactics_sequence:
            if current_state is None or not current_state.goals:
                break
                
            result = dojo.run_tac(current_state, tactic)
            success = result is not None
            proof_steps.append({"tactic": tactic, "success": success})
            
            if success:
                current_state = result
        
        # Afficher le resultat
        proof_complete = current_state is not None and not current_state.goals
        print(f"\nSucces: {proof_complete}")
        print(f"Steps executees: {len(proof_steps)}")
        for step in proof_steps:
            status = "OK" if step["success"] else "FAIL"
            print(f"  {step['tactic']}: {status}")
            
print(f"\n[TIMER] Preuve automatique: {timer.format(timer.stop())}")

PREUVE AUTOMATIQUE

ENABLE_AUTO_PROOF: True

[MODE DEMO] Preuve automatique desactivee (ENABLE_DOJO_DEMO=False)
Raison: Le Dojo est necessaire pour la preuve automatique

Pattern de preuve automatique avec LeanRunner:

    # Avec LeanRunner et Dojo actif:
    result = runner.prove_with_tactics(
        theorem,
        ["intro", "intros", "rfl", "simp", "omega"]
    )

    print(f"Succes: {result['success']}")
    for step in result['steps']:
        print(f"  {step['tactic']}: {'OK' if step['success'] else 'FAIL'}")
    

Exemple de sortie attendue:

    Theorem: hello_world
    Succes: True
    Steps: 1
      rfl: OK
    

[TIMER] Preuve automatique: 1ms


## 9. Depots Avances

LeanDojo peut tracer des depots plus complexes. Voici quelques exemples notables :

### formal-conjectures (Google DeepMind)

Conjectures mathematiques formalisees par l'equipe DeepMind. Contient des problemes ouverts et resolus.

### Mathlib4

La bibliotheque mathematique principale de Lean 4 (~4M lignes). Le tracing complet prend plusieurs heures.

**Note** : Ces depots sont volumineux. Le premier tracing peut prendre 30 minutes a plusieurs heures.

In [20]:
# =============================================================================
# Depots avances disponibles
# =============================================================================
timer.start("Depots avances")
print("=" * 60)
print("DEPOTS AVANCES")
print("=" * 60)

ADVANCED_REPOS = {
    "formal-conjectures": {
        "url": "https://github.com/google-deepmind/formal-conjectures",
        "commit": "ce0a081ab74d625948c44da6022992e1f9db070a",
        "lean_version": "4.22.0",
        "description": "Google DeepMind formalized conjectures",
        "tracing_time": "30-60 min",
    },
    "mathlib4": {
        "url": "https://github.com/leanprover-community/mathlib4",
        "commit": "v4.15.0",
        "lean_version": "4.15.0",
        "description": "Bibliotheque mathematique Lean 4 (~4M lignes)",
        "tracing_time": "2-4 heures",
    },
}

print("\nDepots disponibles pour tests avances:")
print()

for name, info in ADVANCED_REPOS.items():
    print(f"  {name}")
    print(f"    Description: {info['description']}")
    print(f"    Lean version: {info['lean_version']}")
    print(f"    Temps de tracing: {info['tracing_time']}")
    print()
print(f"[TIMER] Depots avances: {timer.format(timer.stop())}")

DEPOTS AVANCES

Depots disponibles pour tests avances:

  formal-conjectures
    Description: Google DeepMind formalized conjectures
    Lean version: 4.22.0
    Temps de tracing: 30-60 min

  mathlib4
    Description: Bibliotheque mathematique Lean 4 (~4M lignes)
    Lean version: 4.15.0
    Temps de tracing: 2-4 heures

[TIMER] Depots avances: 1ms


### Exemple : Tracing de formal-conjectures

Le code ci-dessous montre comment tracer le depot `formal-conjectures`.

**Attention** : Ce tracing telecharge Mathlib4 (~1 Go) et prend 30-60 minutes la premiere fois.

In [21]:
# =============================================================================
# Exemple avec formal-conjectures (decommenter pour executer)
# =============================================================================
print("=" * 60)
print("EXEMPLE FORMAL-CONJECTURES")
print("=" * 60)

print("\n[CODE COMMENTE] Decommentez pour executer")
print("\nCe code effectue les operations suivantes:")
print("  1. Cree une reference au depot formal-conjectures")
print("  2. Trace le depot (telecharge Mathlib4, ~1 Go)")
print("  3. Extrait les theoremes/conjectures")
print("\nTemps estime: 30-60 minutes (premiere fois)")

# Decommentez le code ci-dessous pour executer:
#
# if LEANDOJO_AVAILABLE:
#     deepmind_repo = LeanGitRepo(
#         ADVANCED_REPOS["formal-conjectures"]["url"],
#         ADVANCED_REPOS["formal-conjectures"]["commit"]
#     )
#
#     print(f"Tracing formal-conjectures...")
#     traced_deepmind = trace(deepmind_repo)
#
#     conjectures = list(traced_deepmind.get_traced_theorems())
#     print(f"Found {len(conjectures)} theorems/conjectures")
#
#     for c in conjectures[:10]:
#         # Handle TracedTheorem wrapper
#         inner = c.theorem if hasattr(c, 'theorem') else c
#         name = inner.full_name if hasattr(inner, 'full_name') else str(inner)
#         print(f"  - {name}")

EXEMPLE FORMAL-CONJECTURES

[CODE COMMENTE] Decommentez pour executer

Ce code effectue les operations suivantes:
  1. Cree une reference au depot formal-conjectures
  2. Trace le depot (telecharge Mathlib4, ~1 Go)
  3. Extrait les theoremes/conjectures

Temps estime: 30-60 minutes (premiere fois)


## 10. Integration avec les LLMs

LeanDojo est concu pour l'integration avec des LLMs. Le workflow typique est :

```
1. Extraire les theoremes d'un depot
2. Ouvrir un Dojo pour un theoreme
3. Formater l'etat pour le LLM
4. Envoyer au LLM pour obtenir une tactique
5. Executer la tactique dans le Dojo
6. Repeter jusqu'a ce que la preuve soit complete ou echoue
```

### Format du Prompt

Le LLM recoit l'etat de preuve en format Lean et doit retourner une tactique valide.

In [22]:
# =============================================================================
# Formattage de l'etat de preuve pour un LLM
# =============================================================================
timer.start("Integration LLM")
print("=" * 60)
print("INTEGRATION LLM")
print("=" * 60)

def format_state_for_llm(state):
    """
    Formate l'etat de preuve pour envoi a un LLM.

    Le format est concu pour etre clair et non-ambigu :
    - Affiche l'etat Lean en bloc de code
    - Demande une seule tactique en reponse

    Args:
        state: TacticState de LeanDojo ou objet avec attribut 'pp'

    Returns:
        str: Prompt formate pour le LLM
    """
    prompt = "Given the following Lean 4 proof state:\n\n"

    # Extraire le pretty-print
    pp_state = getattr(state, 'pp', str(state))
    prompt += f"```lean\n{pp_state}\n```\n\n"

    prompt += "Suggest the next tactic to apply. "
    prompt += "Respond with ONLY the tactic, no explanation."

    return prompt

# Demonstration
print("\nExemple de prompt pour un LLM:")
print("-" * 40)

# Creer un etat mock pour la demo
class MockState:
    pp = """n m : Nat
h : n > 0
⊢ n + m = m + n"""

print(format_state_for_llm(MockState()))
print("-" * 40)

print("\nReponse attendue du LLM: 'omega' ou 'ring' ou 'simp [Nat.add_comm]'")
print(f"[TIMER] Integration LLM: {timer.format(timer.stop())}")

INTEGRATION LLM

Exemple de prompt pour un LLM:
----------------------------------------
Given the following Lean 4 proof state:

```lean
n m : Nat
h : n > 0
⊢ n + m = m + n
```

Suggest the next tactic to apply. Respond with ONLY the tactic, no explanation.
----------------------------------------

Reponse attendue du LLM: 'omega' ou 'ring' ou 'simp [Nat.add_comm]'
[TIMER] Integration LLM: 1ms


### Exemple Complet avec LLM (Pseudo-code)

Le code ci-dessous montre le pattern complet d'integration. Dans un cas reel, vous utiliseriez l'API OpenAI ou Anthropic.

Voir **Lean-7-LLM-Integration.ipynb** pour des exemples fonctionnels avec vraies APIs.

In [23]:
# =============================================================================
# Pattern d'integration LLM (pseudo-code)
# =============================================================================
print("=" * 60)
print("PATTERN D'INTEGRATION LLM")
print("=" * 60)

def prove_with_llm(theorem, max_steps=10):
    """
    Pseudo-code pour prouver un theoreme avec un LLM.

    En pratique, remplacez call_llm() par un appel OpenAI/Anthropic.
    """
    proof_steps = []

    with Dojo(theorem) as (dojo, state):
        for step in range(max_steps):
            # 1. Verifier si la preuve est terminee
            if not state.goals:
                return {"success": True, "steps": proof_steps}

            # 2. Formater l'etat pour le LLM
            prompt = format_state_for_llm(state)

            # 3. Appeler le LLM (pseudo-code)
            # tactic = call_llm(prompt)  # OpenAI/Anthropic
            tactic = "simp"  # Placeholder

            # 4. Executer la tactique
            new_state = dojo.run_tac(state, tactic)

            # 5. Enregistrer le resultat
            proof_steps.append({
                "tactic": tactic,
                "success": new_state is not None
            })

            # 6. Gerer l'echec
            if new_state is None:
                return {"success": False, "steps": proof_steps, "error": f"Invalid tactic: {tactic}"}

            state = new_state

    return {"success": False, "steps": proof_steps, "error": "Max steps reached"}

print("\nPattern prove_with_llm() defini.")
print("\nWorkflow:")
print("  1. Ouvrir Dojo")
print("  2. Boucle:")
print("     a. Formater etat -> LLM")
print("     b. LLM -> tactique")
print("     c. Executer tactique")
print("     d. Si echec ou succes, sortir")
print("  3. Retourner resultat")
print("\nVoir Lean-7-LLM-Integration.ipynb pour implementation reelle")

PATTERN D'INTEGRATION LLM

Pattern prove_with_llm() defini.

Workflow:
  1. Ouvrir Dojo
  2. Boucle:
     a. Formater etat -> LLM
     b. LLM -> tactique
     c. Executer tactique
     d. Si echec ou succes, sortir
  3. Retourner resultat

Voir Lean-7-LLM-Integration.ipynb pour implementation reelle


## 11. Bonnes Pratiques et Limitations

### Bonnes Pratiques

| Pratique | Raison |
|----------|--------|
| **Utilisez le cache** | Le tracing est couteux, le cache accelere les executions |
| **Commit specifique** | Utilisez toujours un hash, jamais une branche |
| **GitHub token** | Evitez le rate limiting (60 req/h sans token) |
| **WSL sur Windows** | Le tracing natif Windows peut se bloquer |
| **Python 3.10-3.12** | LeanDojo n'est pas compatible avec Python 3.13+ |

### Limitations Connues

| Limitation | Impact | Contournement |
|------------|--------|---------------|
| **Lean 4 only** | LeanDojo 4.x ne supporte pas Lean 3 | Utiliser LeanDojo 1.x pour Lean 3 |
| **lean-toolchain** | Le depot doit avoir ce fichier | Ajouter manuellement si manquant |
| **Python < 3.13** | Dependances incompatibles | Utiliser conda/venv |
| **Espace disque** | Mathlib4 ~5 Go | Prevoir de l'espace |
| **Windows natif** | Tracing peut bloquer | Utiliser WSL |

### References

- [LeanDojo Documentation](https://leandojo.readthedocs.io/)
- [LeanDojo Paper](https://arxiv.org/abs/2306.15626) (NeurIPS 2023)
- [lean4-example Repository](https://github.com/yangky11/lean4-example)
- [ReProver](https://github.com/lean-dojo/ReProver) - Theorem prover base sur LeanDojo

## Resume

Dans ce notebook, nous avons couvert :

| Section | Contenu |
|---------|---------|
| **1-3** | Installation, configuration, imports |
| **4** | Creation de references de depot (`LeanGitRepo`) |
| **5** | Tracing et cache |
| **6** | Extraction des theoremes |
| **7** | Environnement Dojo et execution de tactiques |
| **8** | Interface simplifiee `lean_runner.py` |
| **9** | Depots avances (formal-conjectures, Mathlib4) |
| **10** | Integration avec les LLMs |

### Points cles a retenir

1. **LeanDojo** permet l'interaction programmatique avec Lean 4
2. Le **tracing** est l'operation centrale (compile et extrait les metadonnees)
3. Le **Dojo** permet d'executer des tactiques une par une
4. Le **cache** (`~/.cache/lean_dojo/`) accelere les executions suivantes
5. Sur **Windows**, utilisez WSL pour eviter les blocages

### Prochaines etapes

- **Lean-7-LLM-Integration.ipynb** : Patterns d'integration LLM-Lean avec vraies APIs
- **Lean-8-Agentic-Proving.ipynb** : Agents autonomes pour la preuve de theoremes

## 12. Resume des Temps d'Execution

Cette cellule affiche un resume de tous les temps mesures pendant l'execution du notebook.

In [24]:
# =============================================================================
# Resume des temps d'execution
# =============================================================================
timer.summary()

RESUME DES TEMPS D'EXECUTION
  Verification environnement: 1ms
  Configuration GitHub: 10ms
  Import LeanDojo: 1.1s
  Creation repo reference: 1.1s
  Verification cache: 363ms
  Tracing: 1.3min
  Extraction theoremes: 4.2s
  Details theoreme: 0ms
  Dojo ouverture: 1ms
  Execution tactiques: 0ms
  Lean Runner: 10ms
  Extraction theoremes lean_runner: 1ms
  Preuve automatique: 1ms
  Depots avances: 1ms
  Integration LLM: 1ms
  TOTAL: 1.4min
